Setup.

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

print(f"Tensorflow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

Load the mnist database.

In [ ]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
image_data = np.load("../data/image_data.npy")
image_data = image_data.reshape((-1, 28, 28))
label_data = np.load("../data/label_data.npy")
print("image_data shape", image_data.shape)

row_indices = np.random.permutation(image_data.shape[0])
train_index = int(0.8 * image_data.shape[0])

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = np.append(x_train, image_data[row_indices[:train_index], :], axis=0)
y_train = np.append(y_train, label_data[row_indices[:train_index]], axis=0)
x_test = np.append(x_test, image_data[row_indices[train_index:], :], axis=0)
y_test = np.append(y_test, label_data[row_indices[train_index:]], axis=0)

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


Build the model

In [ ]:
# model = keras.models.load_model("../model.h5")
# Build the model
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.experimental.preprocessing.Rescaling(1.0 / 255),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Train the model.

In [ ]:
batch_size = 128
epochs = 20

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[tensorboard_callback])

In [ ]:
score = model.evaluate(x_test, y_test)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
model.save("model.h5")
print("Saved to model.h5")